In [1]:
using LinearAlgebra, Plots, LaTeXStrings, Symbolics

In [2]:
const E = 2.1E06
const ν = 0.3

global C = ((1 - ν) / ((1 + ν) * (1 - ν))) * E * [1 (ν/(1-ν)) 0; (ν/(1-ν)) 1 0; 0 0 ((1-2*ν)/(2*(1-ν)))];

In [3]:
const numNodosx = 3
const numNodosy = 3
const numNodosG = numNodosx * numNodosy

global GDLG = numNodosG * 2  # En el caso de elasticidad bidimensional se tiene 2 grados de libertad por nodo

const longx = 3.0
const longy = 3.0

const numElementos = (numNodosx - 1) * (numNodosy - 1);

In [4]:
begin
    @variables r s
    Dr = Differential(r)
    Ds = Differential(s)

    nFI = 4

    h₁ = 1 / 4 * (1 + r) * (1 + s)
    h₂ = 1 / 4 * (1 - r) * (1 + s)
    h₃ = 1 / 4 * (1 - r) * (1 - s)
    h₄ = 1 / 4 * (1 + r) * (1 - s)

    global H = [h₁ h₂ h₃ h₄]
    global ∇H = [transpose([expand_derivatives(Dr(H[i])) for i in 1:nFI]); transpose([expand_derivatives(Ds(H[i])) for i in 1:nFI])]
end

2×4 Matrix{Num}:
 0.25(1 + s)  -0.25(1 + s)  -0.25(1 - s)   0.25(1 - s)
 0.25(1 + r)   0.25(1 - r)  -0.25(1 - r)  -0.25(1 + r)

In [5]:
# Puntos de Gauss para itegración
nNodosInt = 9
GL9Q = [-sqrt(0.6) -sqrt(0.6); 0 -sqrt(0.6); sqrt(0.6) -sqrt(0.6); sqrt(0.6) 0; sqrt(0.6) sqrt(0.6);
    0 sqrt(0.6); -sqrt(0.6) sqrt(0.6); -sqrt(0.6) 0; 0 0]

wGL9Q = [5/9 5/9; 8/9 5/9; 5/9 5/9; 5/9 8/9; 5/9 5/9; 8/9 5/9; 5/9 5/9; 5/9 8/9; 8/9 8/9];

In [6]:
# Matriz de conectividad global para elementos bidimensionales de cuatro nodos 
#(El numero de elementos puede variar)
function matrizConectividadGlobal(numNodosx, numNodosy, numElementos)

    matrizConectividad = zeros(Int, numElementos, 4)
    elemento = 1

    for j in 1:numNodosy-1
        for i in 1:numNodosx-1
            nodoInferiorIzquierdo::Int = i + ((j - 1) * numNodosx)
            matrizConectividad[elemento, :] = [(nodoInferiorIzquierdo + numNodosx + 1) (nodoInferiorIzquierdo + numNodosx) nodoInferiorIzquierdo (nodoInferiorIzquierdo + 1)]
            elemento += 1
        end
    end

    return matrizConectividad
end

matrizConectividadGlobal (generic function with 1 method)

In [7]:
function mallador(numNodosx, numNodosy, longx, longy, numElementos)

    numSegmentosx = numNodosx - 1
    numSegmentosy = numNodosy - 1
    dNodosx = longx / numSegmentosx
    dNodosy = longy / numSegmentosy
    numNodosG = numNodosx * numNodosy
    matrizCoordG = zeros(numNodosG, 2)
    Coordy = 0
    nodo = 1

    for j in 1:numNodosy
        Coordx = 0
        for i in 1:numNodosx
            matrizCoordG[nodo, :] = [Coordx Coordy]
            Coordx += dNodosx
            nodo += 1
        end
        Coordy += dNodosy
    end

    return matrizCoordG
end

mallador (generic function with 1 method)

In [8]:
function matrizGdGL(numNodosx, numNodosy, mConectividadGlobal)

    numElementos = (numNodosx - 1) * (numNodosy - 1)
    numNodosGlobales = numNodosx * numNodosy
    mGdGL = zeros(Integer, 8, numElementos)

    for i in 1:numElementos
        j = 1
        while j <= 4
            zz = mConectividadGlobal[i, :]
            mGdGL[2*j-1, i] = zz[j] * 2 - 1
            mGdGL[2*j, i] = zz[j] * 2
            j += 1
        end
    end

    return mGdGL
end

matrizGdGL (generic function with 1 method)

In [9]:
begin
    mConectG = matrizConectividadGlobal(numNodosx, numNodosy, numElementos)
    #display(mConectG)
    mGdlG = matrizGdGL(numNodosx, numNodosy, mConectG)
    display(mGdlG)
    mGCoorNodales = mallador(numNodosx, numNodosy, longx, longy, numElementos)
    #display(mGCoorNodales)
end

8×4 Matrix{Integer}:
  9  11  15  17
 10  12  16  18
  7   9  13  15
  8  10  14  16
  1   3   7   9
  2   4   8  10
  3   5   9  11
  4   6  10  12

9×2 Matrix{Float64}:
 0.0  0.0
 1.5  0.0
 3.0  0.0
 0.0  1.5
 1.5  1.5
 3.0  1.5
 0.0  3.0
 1.5  3.0
 3.0  3.0

In [10]:
function Jacobiano(mGCoorNodales, mConectG, el)
    mNodalCoords = zeros(4, 2)
    for nodo in 1:4
        mNodalCoords[nodo, :] = mGCoorNodales[mConectG[el, :][nodo], :]
    end

    x₀ = expand((H*mNodalCoords[:, 1])[1])
    y₀ = expand((H*mNodalCoords[:, 2])[1])

    dxdr = expand_derivatives(Dr(x₀))
    dxds = expand_derivatives(Ds(x₀))
    dydr = expand_derivatives(Dr(y₀))
    dyds = expand_derivatives(Ds(y₀))

    J = [dxdr dydr; dxds dyds] # Jacobiano

    return J
end

Jacobiano (generic function with 1 method)

In [11]:
function BL0(∇H, J)
 
    B = inv(J) * ∇H

    BL0₁ = [B[1, 1] 0 B[1,2] 0 B[1,3] 0 B[1,4] 0]
    BL0₂ = [0 B[2,1] 0 B[2, 2] 0 B[2,3] 0 B[2,4]]
    BL0₃ = [B[2,1] B[1, 1] B[2, 2] B[1,2] B[2,3] B[1,3] B[2,4] B[1,4]]

    BL0 = vcat(BL0₁, BL0₂, BL0₃)

    return BL0
end

BL0 (generic function with 1 method)

In [12]:
function BL1(∇H, J, Uₑ)
    
    B = inv(J) * ∇H

    l₁₁, l₁₂, l₂₁, l₂₂ = 0, 0, 0, 0

    for k in 1:4 l₁₁ += B[1, k] * Uₑ[k, 1] end
    for k in 1:4 l₁₂ += B[2, k] * Uₑ[k, 1] end
    for k in 1:4 l₂₁ += B[1, k] * Uₑ[k, 2] end
    for k in 1:4 l₂₂ += B[2, k] * Uₑ[k, 2] end

    BL1₁ = [l₁₁ * B[1, 1] l₂₁ * B[1, 1] l₁₁ * B[1, 2] l₂₁ * B[1, 2] l₁₁ * B[1, 3] l₂₁ * B[1, 3] l₁₁ * B[1, 4] l₂₁ * B[1, 4]]
    BL1₂ = [l₁₂ * B[2, 1] l₂₂ * B[2, 1] l₁₂ * B[2, 2] l₂₂ * B[2, 2] l₁₂ * B[2, 3] l₂₂ * B[2, 3] l₁₂ * B[2, 4] l₂₂ * B[2, 4]]
    BL1₃ = [(l₁₁ * B[2, 1] + l₁₂ * B[1, 1]) (l₂₁ * B[2, 1] + l₂₂ * B[1, 1]) (l₁₁ * B[2, 2] + l₁₂ * B[1, 2]) (l₂₁ * B[2, 2] + l₂₂ * B[1, 2]) (l₁₁ * B[2, 3] + l₁₂ * B[1, 3]) (l₂₁ * B[2, 3] + l₂₂ * B[1, 3]) (l₁₁ * B[2, 4] + l₁₂ * B[1, 4]) (l₂₁ * B[2, 4] + l₂₂ * B[1, 4])]
    
    BL1 = vcat(BL1₁, BL1₂, BL1₃)

    return BL1
end

BL1 (generic function with 1 method)

In [31]:
# TDG -> Tensor Gradiente de Deformaciones
function TGD(Uₑ, J, mGCoorNodales, mConectG, el) # TDG -> TENSOR GRADIENTE DE DEFORMACIÓN
    X₁ = inv(J)
    mNodalCoordsₜ = zeros(4, 2)
    for nodo in 1:4
        mNodalCoordsₜ[nodo, :] = mGCoorNodales[mConectG[el, :][nodo], :] + Uₑ[nodo, :]
    end

    xₜ = expand((H*mNodalCoordsₜ[:, 1])[1])
    yₜ = expand((H*mNodalCoordsₜ[:, 2])[1])

    dxdrₜ = expand_derivatives(Dr(xₜ))
    dxdsₜ = expand_derivatives(Ds(xₜ))
    dydrₜ = expand_derivatives(Dr(yₜ))
    dydsₜ = expand_derivatives(Ds(yₜ))

    X₂ = [dxdrₜ dydrₜ; dxdsₜ dydsₜ]

    X = X₂ * X₁ # X -> TENSOR GRADIENTE DE DEFORMACIONES
    return X

end

TGD (generic function with 1 method)

In [14]:
# Matriz de Rigidez Global Lineal *******
function KGL0(mConectG, mGCoorNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q)

    KGL0 = zeros(Float64, GDLG, GDLG)

    for el in 1:numElementos
        KEL = zeros(Float64, 8, 8)
        GLE = mGdlG[:, el]
        numGLE = length(GLE)
        
        J = Jacobiano(mGCoorNodales, mConectG, el)
        BL₀ = BL0(∇H, J)
        
        for i in 1:9 # Integración en los puntos de Gauss
            Jg_det = substitute(det(J), Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            Bg = substitute(BL₀, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            KEL += (transpose(Bg) * C * Bg * Jg_det * wGL9Q[i, 1] * wGL9Q[i, 2])
        end

        for i in 1:numGLE
            for j in 1:numGLE
                KGL0[GLE[i], GLE[j]] += KEL[i, j].val
            end
        end
    end
    return KGL0
end

KGL0 (generic function with 1 method)

In [15]:
#RE = zeros(Number, GDLGlobales, numElementos)
RG = zeros(GDLG)
RG[18] = 3000
RG[14] = 3000
#RG[8] = 3000
#RG[6] = 3000
#RG = [0., 3000., 0., 3000., 0., 0., 0., 0.]
#display(RG)

3000

In [16]:
KGL = KGL0(mConectG, mGCoorNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q);

In [17]:
# Caso 1 (1 elemento)

# Imponemos condiciones de contorno de Dirichlet
#=
# U1x=0 => U1=0
KG[1, :] .= 0
KG[:, 1] .= 0
KG[1, 1] = 1

# U1y=0 => U2=0
KG[2, :] .= 0
KG[:, 2] .= 0
KG[2, 2] = 1

# U2x=0 => U3=0
KG[3, :] .= 0
KG[:, 3] .= 0
KG[3, 3] = 1

# U2y=0 => U4=0
KG[4, :] .= 0
KG[:, 4] .= 0
KG[4, 4] = 1

display(KG)

U₀ = KGL \ RG # U₀ -> Desplazamientos en ⁰U 
=#

In [18]:
# Caso 2 (4 Elementos)
# Imponemos condiciones de contorno de Dirichlet

#
# U1x=0 => U1=0
KGL[1, :] .= 0
KGL[:, 1] .= 0
KGL[1, 1] = 1

# U1y=0 => U2=0
KGL[2, :] .= 0
KGL[:, 2] .= 0
KGL[2, 2] = 1

# U2x=0 => U3=0
KGL[3, :] .= 0
KGL[:, 3] .= 0
KGL[3, 3] = 1

# U2y=0 => U4=0
KGL[4, :] .= 0
KGL[:, 4] .= 0
KGL[4, 4] = 1

# U3x=0 => U5=0
KGL[5, :] .= 0
KGL[:, 5] .= 0
KGL[5, 5] = 1

# U3y=0 => U6=0
KGL[6, :] .= 0
KGL[:, 6] .= 0
KGL[6, 6] = 1

1

In [19]:
U = KGL \ RG

18-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.00038012514028317975
  0.0024876583744940484
 -3.8883276355802194e-19
  0.0013895381141987445
 -0.00038012514028318035
  0.002487658374494048
  0.0023806785862530177
  0.007497264615256223
  8.325420909912773e-20
  0.0012774191848092266
 -0.0023806785862530177
  0.007497264615256221

In [32]:
# mGdlG <- matrix Grados de libertad global
function FGA(U)
    FG = zeros(Float64, GDLG)

    for el in 1:numElementos
        mGdlE = mGdlG[:, el]
        numGLE = length(mGdlE)
        
        Uₑ = transpose(reshape(U[mGdlE], 2, 4))
        Fₑ = zeros(8, 1)

        J = Jacobiano(mGCoorNodales, mConectG, el)

        BL₀ = BL0(∇H, J)
        BL₁ = BL1(∇H, J, Uₑ)
        BL = BL₀ + BL₁
        
        X = TGD(Uₑ, J, mGCoorNodales, mConectG, el) # TENSOR GRADIENTE DE DEFORMACIONES
    
        ϵ = 1 / 2 * (transpose(X) * X - Matrix(I, 2, 2)) # TENSOR DE DEFORMACIONES DE GREEN-LAGRANGE

        ϵv = [ϵ[1, 1]; ϵ[2, 2]; ϵ[1, 2]] # VECTOR DE DEFORMACIONES DE GREEN-LAGRANGE

        S = C * ϵv # S -> SEGUNDO TENSOR DE PIOLA KIRCHHOFF
        for i in 1:9
            Sₑₗ = substitute(S, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            BLₑ = substitute(BL, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            Fₑ += transpose(BLₑ) * Sₑₗ
        end

        for i in 1:numGLE # numGLE -> Grados de libertad del elemento
            FG[mGdlE[i]] += Fₑ[i]
        end
    end
    return FG
end

FGA (generic function with 1 method)

In [33]:
FGA(U)

18-element Vector{Float64}:
  -2188.4754768765183
  -6660.360106838713
      3.319655661471188e-10
 -10730.465803575866
   2188.47547687629
  -6660.360106838456
  -1162.1609818955326
  -1626.3907263619958
      8.833467290969566e-10
   3113.851689201206
   1162.1609818941374
  -1626.3907263621768
   1312.0073928475242
  11061.668975738656
      1.709850039333105e-10
   2066.7778292984503
  -1312.0073928472862
  11061.668975738889

In [21]:
function KGL1(mConectG, mGCoorNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q, U)
    KGL = zeros(Float64, GDLG, GDLG)

    for el in 1:numElementos
        KEL = zeros(Float64, 8, 8)
        mGdlE = mGdlG[:, el]
        numGLE = length(mGdlE)
        Uₑ = transpose(reshape(U[mGdlE], 2, 4))
        
        J = Jacobiano(mGCoorNodales, mConectG, el)
        BL₀ = BL0(∇H, J)
        BL₁ = BL1(∇H, J, Uₑ)
        BL = BL₀ + BL₁
        
        for i in 1:9 # Integración en los puntos de Gauss
            Jg_det = substitute(det(J), Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            Bg = substitute(BL, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            KEL += (transpose(Bg) * C * Bg * Jg_det * wGL9Q[i, 1] * wGL9Q[i, 2])
        end

        for i in 1:numGLE
            for j in 1:numGLE
                KGL[mGdlE[i], mGdlE[j]] += KEL[i, j].val
            end
        end
    end
    return KGL
end

KGL1 (generic function with 1 method)

In [22]:
KGL1(mConectG, mGCoorNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q, U)

18×18 Matrix{Float64}:
 692298.0             2.88812e5  …       0.0              0.0
      2.88812e5       6.94155e5          0.0              0.0
     -4.61514e5  -57712.2                0.0              0.0
  57811.8             1.15683e5          0.0              0.0
      0.0             0.0                0.0              0.0
      0.0             0.0        …       0.0              0.0
      1.15355e5   57619.4                0.0              0.0
 -57913.8            -4.63e5             0.0              0.0
     -3.46139e5      -2.8872e5          -3.45133e5  -289324.0
     -2.8871e5       -3.46838e5         -2.89233e5       -3.48697e5
      0.0             0.0        …       1.15172e5    58600.5
      0.0             0.0           -56932.0             -4.64296e5
      0.0             0.0                0.0              0.0
      0.0             0.0                0.0              0.0
      0.0             0.0               -4.60203e5   -59544.8
      0.0             0.0        … 

In [23]:
function KGNL(mConectG, mGCoorNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q, U)
    KGNL = zeros(Float64, GDLG, GDLG)
    for el in 1:numElementos
        mGdlE = mGdlG[:, el]
        numGLE = length(mGdlE)
        KENL = zeros(numGLE, numGLE)
        Uₑ = transpose(reshape(U[mGdlE], 2, 4))

        J = Jacobiano(mGCoorNodales, mConectG, el)
        B = inv(J) * ∇H
        
        X = TGD(Uₑ, J, mGCoorNodales, mConectG, el) # TENSOR GRADIENTE DE DEFORMACIONES    
        ϵ = 1 / 2 * (transpose(X) * X - Matrix(I, 2, 2)) # TENSOR DE DEFORMACIONES DE GREEN-LAGRANGE
        ϵv = [ϵ[1, 1]; ϵ[2, 2]; ϵ[1, 2]] # VECTOR DE DEFORMACIONES DE GREEN-LAGRANGE
        S = C * ϵv # S -> SEGUNDO TENSOR DE PIOLA KIRCHHOFF
        ST = [S[1,1] S[3,1] 0 0; S[3,1] S[2,1] 0 0; 0 0 S[1,1] S[3,1]; 0 0 S[3,1] S[2,1]]

        BNL1 = [B[1,1] 0 B[1,2] 0 B[1,3] 0 B[1,4] 0]
        BNL2 = [B[2,1] 0 B[2,2] 0 B[2,3] 0 B[2,4] 0]
        BNL3 = [0 B[1,1] 0 B[1,2] 0 B[1,3] 0 B[1,4]]
        BNL4 = [0 B[2,1] 0 B[2,2] 0 B[2,3] 0 B[2,4]]
        BNL = vcat(BNL1, BNL2, BNL3, BNL4)

        for i in 1:9 # Integración en los puntos de Gauss
            BNLG = substitute(BNL, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            STG = substitute(ST, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            KENL += (transpose(BNLG) * STG * BNLG * wGL9Q[i, 1] * wGL9Q[i, 2])
        end

        for i in 1:numGLE
            for j in 1:numGLE
                KGNL[mGdlE[i], mGdlE[j]] += KENL[i, j]
            end
        end
        
    end

    return KGNL
    
end

KGNL (generic function with 1 method)

In [24]:
KGNL(mConectG, mGCoorNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q, U)

18×18 Matrix{Float64}:
  1816.13       0.0      114.423  …      0.0        0.0        0.0
     0.0     1816.13       0.0           0.0        0.0        0.0
   114.423      0.0     3026.64          0.0        0.0        0.0
     0.0      114.423      0.0           0.0        0.0        0.0
     0.0        0.0      114.423         0.0        0.0        0.0
     0.0        0.0        0.0    …      0.0        0.0        0.0
 -1181.89       0.0     -846.753         0.0        0.0        0.0
     0.0    -1181.89       0.0        -128.249      0.0        0.0
  -748.664      0.0    -1561.98          0.0     -853.737      0.0
     0.0     -748.664      0.0        -719.543      0.0     -853.737
     0.0        0.0     -846.753  …      0.0    -2229.94       0.0
     0.0        0.0        0.0        -128.249      0.0    -2229.94
     0.0        0.0        0.0           0.0        0.0        0.0
     0.0        0.0        0.0        1167.18       0.0        0.0
     0.0        0.0        0.0      

In [49]:
it = 0
KG = zeros(GDLG, GDLG)
RG = zeros(GDLG)
RG[18] = 3000
RG[14] = 3000
RA = zeros(GDLG)
Ui = U
ΔU = 0
#=
KG = KGL1(mConectG, mGCoorNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q, Ui) + KGNL(mConectG, mGCoorNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q, Ui)
display(KG)
display(RG)
RA = RG - FGA(Ui)
display(RA)
=#
while it <= 100
    #display(Ui)
    display(RG - FGA(Ui))
    KG = KGL1(mConectG, mGCoorNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q, Ui) + KGNL(mConectG, mGCoorNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q, Ui)
    RA = RG - FGA(Ui)
    RA = zeros(GDLG)
    RA[18] = 3000
    RA[14] = 3000
    #display(RA)

    KG[1, :] .= 0
    KG[:, 1] .= 0
    KG[1, 1] = 1

    # U1y=0 => U2=0
    KG[2, :] .= 0
    KG[:, 2] .= 0
    KG[2, 2] = 1

    # U2x=0 => U3=0
    KG[3, :] .= 0
    KG[:, 3] .= 0
    KG[3, 3] = 1

    # U2y=0 => U4=0
    KG[4, :] .= 0
    KG[:, 4] .= 0
    KG[4, 4] = 1

    # U3x=0 => U5=0
    KG[5, :] .= 0
    KG[:, 5] .= 0
    KG[5, 5] = 1

    # U3y=0 => U6=0
    KG[6, :] .= 0
    KG[:, 6] .= 0
    KG[6, 6] = 1   
    #display(KG)

    ΔU = KG \ RA
    Ui +=  ΔU    # OJO REVISAR CUIDADOSAMENTE
it += 1
end

18-element Vector{Float64}:
  2188.4754768765183
  6660.360106838713
    -3.319655661471188e-10
 10730.465803575866
 -2188.47547687629
  6660.360106838456
  1162.1609818955326
  1626.3907263619958
    -8.833467290969566e-10
 -3113.851689201206
 -1162.1609818941374
  1626.3907263621768
 -1312.0073928475242
 -8061.668975738656
    -1.709850039333105e-10
 -2066.7778292984503
  1312.0073928472862
 -8061.668975738889

18-element Vector{Float64}:
   4369.3344700830485
  13320.525461319892
     -9.094947017729282e-12
  21438.090501860504
  -4369.33447008304
  13320.525461319921
   2323.308516386266
   3270.6564341907833
     -1.0322764865122736e-10
  -6226.927254300072
  -2323.3085163862684
   3270.656434191611
  -2656.8392856243313
 -19106.6374129959
     -1.7007550923153758e-10
  -4180.252212590087
   2656.8392856246073
 -19106.637412996657

18-element Vector{Float64}:
   6542.698431603245
  19980.550816818708
      7.885319064371288e-10
  32123.090004603022
  -6542.698431603931
  19980.55081681904
   3483.436496196923
   4932.747993431738
      2.1582309273071587e-9
  -9339.265593394974
  -3483.436496199083
   4932.747993431596
  -4033.915645112003
 -30135.067618012115
      3.1650415621697903e-10
  -6340.28679568449
   4033.915645111584
 -30135.067618012516

18-element Vector{Float64}:
   8708.686925176095
  26640.49034260326
      7.894414011389017e-10
  42785.673973080666
  -8708.68692517678
  26640.490342603614
   4642.539440030979
   6612.615035329152
      1.889929990284145e-9
 -12450.906253452473
  -4642.539440033073
   6612.615035329167
  -5442.675570280627
 -41147.11432854969
      1.1295924196019769e-9
  -8546.74981839361
   5442.675570279601
 -41147.11432855009

18-element Vector{Float64}:
  10867.417686109431
  33300.39761312909
      2.764863893389702e-10
  53426.04607638767
 -10867.417686109813
  33300.39761312916
   5800.612448625881
   8310.206088756138
     -6.421032594516873e-10
 -15561.889361931255
  -5800.612448625229
   8310.206088755505
  -6882.576318321364
 -52142.92510940668
     -2.3646862246096134e-10
 -10799.513899413723
   6882.576318321698
 -52142.92510940589

...

18-element Vector{Float64}:
  190627.45525986547
  651183.4203618722
       0.0
  953954.7922622492
 -190627.45525986544
  651183.420361872
  108010.85119591115
  232549.71445059066
      -2.473825588822365e-10
 -304681.4314473193
 -108010.85119591077
  232549.71445058996
 -228589.03539772925
      -1.0028076163381257e6
      -2.9103830456733704e-10
 -405124.3977636048
  228589.0353977294
      -1.0028076163381247e6

18-element Vector{Float64}:
  192485.4749350269
  657994.5867686529
       0.0
  963008.0885493524
 -192485.4749350269
  657994.5867686529
  109078.76300439713
  235637.38590046437
       3.41970007866621e-10
 -307916.410187989
 -109078.76300439725
  235637.38590046496
 -231644.42711881053
      -1.0125116501094176e6
       1.1641532182693481e-10
 -411332.32348076266
  231644.42711881016
      -1.0125116501094188e6

18-element Vector{Float64}:
  194343.46135096689
  664808.2426981002
      -4.0745362639427185e-10
  972047.4026320002
 -194343.46135096648
  664808.2426980993
  110145.87342244963
  238736.9024109284
      -3.055902197957039e-10
 -311154.6172848732
 -110145.87342244915
  238736.90241092967
 -234708.25328211463
      -1.0222023390121844e6
      -1.1641532182693481e-10
 -417578.397540816
  234708.2532821146
      -1.0222023390121843e6

18-element Vector{Float64}:
  196201.47269857148
  671624.389745534
       4.3655745685100555e-10
  981072.7852436713
 -196201.47269857192
  671624.3897455335
  111212.18761141079
  241848.18381514843
       1.0331859812140465e-9
 -314396.08946187224
 -111212.18761141221
  241848.18381515075
 -237780.35890103248
      -1.0318796872431212e6
       2.3283064365386963e-10
 -423862.4684169216
  237780.35890103268
      -1.0318796872431223e6

18-element Vector{Float64}:
  198059.56694561886
  678443.0289455972
       2.9103830456733704e-11
  990084.2872364572
 -198059.56694561892
  678443.0289455975
  112277.71079452193
  244971.149252675
       3.310560714453459e-10
 -317640.8632529323
 -112277.71079452231
  244971.14925267594
 -240860.58915094583
      -1.0415436993869257e6
       8.731149137020111e-10
 -430184.3816062182
  240860.58915094496
      -1.041543699386927e6